In [5]:
import pandas as pd
import pandas.tseries.holiday

In [2]:
data = pd.read_csv('./data-20191112T1252-structure-20191112T1247.csv')
data.loc[22];

In [3]:
def gettimestamps(d):
    result = []
    extras = []
    year = d[0]
    for month in range(1, 13):
        days = d[month]
        for day in days.split(','):
            if day[-1] in '*+':
                extras.append(day[-1])
                day = day[:-1]
            else:
                extras.append(None)
            result.append(pd.Timestamp(year, month, int(day)))
    return result, extras

In [6]:
timestamps = [gettimestamps(data.loc[i]) for i in range(len(data))]
timestamps = pd.DataFrame({
    'timestamps': [ts for ts, _ in timestamps],
    'extras': [extra for _, extra in timestamps]
}, index=range(1999, 2026))

RULES = []
for i in timestamps.index:
    RULES.extend(
        pd.tseries.holiday.Holiday(
            str(ts),
            month=ts.month,
            year=ts.year,
            day=ts.day,
        )
        for j, ts in enumerate(timestamps.loc[i, 'timestamps'])
        if timestamps.loc[i, 'extras'][j] != '*'
    )

class RussianBusinessCalendar(pd.tseries.holiday.AbstractHolidayCalendar):
    rules = RULES

cal = RussianBusinessCalendar()
offset = pd.tseries.offsets.CustomBusinessDay(calendar=cal)

In [7]:
df = pd.DataFrame(
    {},
    index=pd.date_range(pd.Timestamp(1999, 1, 1), pd.Timestamp(2025, 12, 31)),
    columns=[
        'выходной', 'праздник', 'предпраздничный рабочий день',
        'последний день месяца', 'последний рабочий день месяца',
        'пн', 'вт', 'ср', 'чт', 'пт', 'сб', 'вс',
        'январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль',
        'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь',
        'Предновогодний выходной день'
    ]
)

df.loc[:, :] = 0

In [8]:
df.loc[cal.holidays(df.index[0], df.index[-1]), 'выходной'] = 1

In [9]:
for year in timestamps.index:
    tss = timestamps.loc[year, 'timestamps']
    extras = timestamps.loc[year, 'extras']
    df.loc[[ts for j, ts in enumerate(tss) if extras[j] == '*'], 'предпраздничный рабочий день'] = 1

In [15]:
for day in df.index:
    if day.is_month_end:
        df.loc[day, 'последний день месяца'] = 1
    df.loc[day, df.columns[day.day_of_week + 5]] = 1
    df.loc[day, df.columns[day.month + 11]] = 1
    n = offset.apply(day)
    if df.loc[day, 'выходной']:
        if n.year != day.year:
            df.loc[day, 'Предновогодний выходной день'] = 1
    elif n.month != day.month:
        df.loc[day, 'последний рабочий день месяца'] = 1

In [16]:
df.loc[pd.date_range(pd.Timestamp(2021, 1, 1), pd.Timestamp(2021, 12, 31))]

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,...,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день
2021-01-01,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2021-01-02,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-01-03,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-01-04,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-01-05,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2021-12-28,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2021-12-29,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2021-12-30,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [17]:
data.loc[17]

Год/Месяц                                                                                   2016
Январь                                                    1,2,3,4,5,6,7,8,9,10,16,17,23,24,30,31
Февраль                                                             6,7,13,14,20*,21,22,23,27,28
Март                                                                   5,6,7,8,12,13,19,20,26,27
Апрель                                                                   2,3,9,10,16,17,23,24,30
Май                                                                1,2,3,7,8,9,14,15,21,22,28,29
Июнь                                                                    4,5,11,12,13,18,19,25,26
Июль                                                                  2,3,9,10,16,17,23,24,30,31
Август                                                                     6,7,13,14,20,21,27,28
Сентябрь                                                                   3,4,10,11,17,18,24,25
Октябрь                       